In [1]:
import os, warnings; warnings.filterwarnings('ignore')

import tensorflow as tf

import keras_tuner as kt

In [2]:
import numpy as np

from tensorflow.keras import layers, models, datasets, optimizers

(train_image, train_label), (test_image, test_label) = datasets.mnist.load_data()

In [3]:
train_image = tf.cast(train_image, tf.float32) / 255.0

In [4]:
test_image = tf.cast(test_image, tf.float32) / 255.0

np.min(test_image[0]), np.max(test_image[0])

(0.0, 1.0)

In [5]:
from tensorflow.keras.models import Sequential

loss = 'sparse_categorical_crossentropy'

def model_builder():
  """
  Model Builder 0
  """
  # Model Builder
  model = Sequential()
  model.add(layers.Flatten(input_shape=(28, 28)))
  model.add(layers.Dense(48, activation='relu'))
  model.add(layers.Dense(10, activation='softmax'))
  # Model Compile
  model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
  # Return Model
  return model

model = model_builder()

hist = model.fit(train_image, train_label, validation_data=(test_image, test_label), epochs=5, verbose=False)

In [6]:
model.evaluate(train_image, train_label, verbose=False)

[0.07605046033859253, 0.9775333404541016]

In [7]:
model.evaluate(test_image, test_label, verbose=False)

[0.11200111359357834, 0.9668999910354614]

In [8]:
def model_builder(hp):
  """
  Model Builder Keras Tuner
  """
  model = Sequential()
  unit_option = hp.Int('units', min_value=16, max_value=64, step=16)
  model.add(layers.Flatten(input_shape=(28, 28)))
  model.add(layers.Dense(unit_option, activation='relu'))
  model.add(layers.Dense(10, activation='softmax'))
  lr_option = hp.Choice('learning_rate', [0.001, 0.01, 0.005])
  optim = optimizers.Adam(learning_rate=lr_option)
  model.compile(optimizer=optim, loss=loss, metrics=['accuracy'])
  return model

project = 'intro_to_kt'

dirt = 'tunerfolder'; target = 'val_accuracy'

tuner = kt.Hyperband(model_builder, objective=target, max_epochs=8, directory=dirt, project_name=project)

In [ ]:
tuner.search(train_image, train_label, epochs=5, validation_data=(test_image, test_label))

In [10]:
optimal = tuner.get_best_hyperparameters()[0]

optimal.get('units')

64

In [11]:
optimal.get('learning_rate')

0.001

In [12]:
optimal_model = tuner.hypermodel.build(optimal)

hist = optimal_model.fit(train_image, train_label, epochs=5, validation_data=(test_image, test_label), verbose=False)

In [13]:
optimal_model.evaluate(train_image, train_label, verbose=False)

[0.05403641238808632, 0.9836833477020264]

In [14]:
optimal_model.evaluate(test_image, test_label, verbose=False)

[0.08981950581073761, 0.9711999893188477]